In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html.read(), 'html.parser')
namelist = bs.find_all('span', {'class': 'green'})

for name in namelist:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


In [12]:
namelist = bs.find_all(string='the prince')
print(len(namelist))

7


In [14]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


In [15]:
for sibling in bs.find('table', {'id': 'giftList'}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [16]:
print(bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())


$15.00



In [17]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img', {'src': re.compile('..\/img\/gifts/img.*.jpg')})

for image in images:
    print(image['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


In [26]:
print(bs.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?'))
print(bs.find_all('span', string='Or maybe he\'s only resting?'))

[<span class="excitingNote">Or maybe he's only resting?</span>]
[<span class="excitingNote">Or maybe he's only resting?</span>]


In [18]:
import random
from urllib.error import HTTPError
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup


def get_internal_links(bs, url):
    '''Retrieves a list of all Internal links found on a page'''
    netloc = urlparse(url).netloc
    scheme = urlparse(url).scheme
    internal_links = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc == '':
            internal_links.add(f'{scheme}://{netloc}/{link.attrs["href"].strip("/")}')
        elif parsed.netloc == netloc:
            internal_links.add(link.attrs['href'])
    return list(internal_links)
            

def get_external_links(bs, url):
    '''Retrieves a list of all external links found on a page'''
    netloc = urlparse(url).netloc
    external_links = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc != '' and parsed.netloc != netloc:
            external_links.add(link.attrs['href'])
    return list(external_links)

def get_random_external_link(starting_page):
    bs = BeautifulSoup(urlopen(starting_page), 'html.parser')
    external_links = get_external_links(bs, starting_page)
    if not len(external_links):
        print('No external links, looking around the site for one')
        internal_links = get_internal_links(bs, starting_page)
        return get_random_external_link(random.choice(internal_links))
    else:
        return random.choice(external_links)
    
def follow_external_only(starting_site):
    try:
        external_link = get_random_external_link(starting_site)
        print(f'Random external link is: {external_link}')
        follow_external_only(external_link)
    except ValueError as e:
        print('Error value link: ', e)
    except HTTPError as e:
        print('Error http: ', e)
    except TimeoutError as e:
        print('Connection timeout: ', e)
    except IndexError as e:
        print('Range index not found: ', e)


follow_external_only('https://pythonworld.ru/')

Random external link is: https://pyplanet.ru/article/telegram-bot-aiogram-quickstart.html
Random external link is: https://zallj.com/g/qqb62fps0r99b979a60661cec322b98fba714163/?erid=5jtCeReNwxHpfQTDve31wmc
Random external link is: https://yandex.cloud/ru/services/smartcaptcha?utm_source=captcha&utm_medium=chbx&utm_campaign=security
Random external link is: https://yandex.ru/legal/cloud_termsofuse
Random external link is: https://yandex.cloud
Random external link is: https://sourcecraft.dev/
Random external link is: https://ya.ru
Random external link is: https://yandex.ru/support/smart-captcha/problems.html?form-unique_key=7639461546662057535&form-fb-hint=2.47
Error http:  HTTP Error 404: Not Found


In [ ]:
# Collects a list of all external URLs found on the site
all_ext_links = []
all_int_links = []


def get_all_external_links(url):
    bs = BeautifulSoup(urlopen(url), 'html.parser')
    internal_links = get_internal_links(bs, url)
    external_links = get_external_links(bs, url)
    for link in external_links:
        if link not in all_ext_links:
            all_ext_links.append(link)
            print(link)

    for link in internal_links:
        if link not in all_int_links:
            all_int_links.append(link)
            get_all_external_links(link)


all_int_links.append('https://oreilly.com')
get_all_external_links('https://www.oreilly.com/')

In [27]:
import requests

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        

def get_page(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

def scrape(url):
    bs = get_page(url)
    title = bs.find('h1').text
    lines = bs.select('div.StoryBodyCompanionColumn div p')
    body = '\n'.join(line.text for line in lines)
    return Content(url, title, body)


url = 'https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/'

content = scrape(url)
print(f'Title: {content.title}')
print(f'URL: {content.url}\n')
print(content.body)

Title: Delivering inclusive urban access: 3 uncomfortable truths
URL: https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/




In [64]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
    
    def print_info(self):
        print('TITLE:', f'{self.title}'.replace("\n", "").replace(" ", ""))
        print(f'URL: {self.url}')
        print('BODY:', f'{self.body}'.replace("\n", "").strip("\t"))

def scrapeCNN(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'headline__wrapper'}).text
    print('body: ')
    print(body)
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'label-large'}).text
    return Content(url, title, body)

url = 'https://www.brookings.edu/research/robotic-rulemaking/'
content = scrapeBrookings(url)
content.print_info()
print()
url = 'https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html'
content = scrapeCNN(url)
content.print_info()

TITLE: Roboticrulemaking
URL: https://www.brookings.edu/research/robotic-rulemaking/
BODY: More On

body: 



      Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu
    

TITLE: DogecoinjumpsafterElonMuskreplacesTwitterbirdwithShibaInu
URL: https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html
BODY:       Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu    


In [76]:
import requests
from bs4 import BeautifulSoup as BS

class Content:
    """Общий родительский класс для всех статей/страниц."""
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print_info(self):
        """Гибкая функция печати, управляющая выводом данных."""
        print(f'URL: {self.url}')
        print(f'TITLE: {self.title}')
        print(f'BODY:\n{self.body}')
        

class Website:
    """Содержит информацию о структуре сайта."""
    def __init__(self, name, url, title_tag, body_tag):
        self.name = name
        self.url = url
        self.title_tag = title_tag
        self.body_tag = body_tag
        

class Crawler:
    def get_page(self, url):
        try:
            response = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BS(response.text, 'html.parser')
    
    def safe_get(self, bs, selector):
        """
        Служебная функция, используемая для получения строки 
        содержимого из объекта BeautifulSoup и селектора.
        Если объект для данного селектора не найден,
        то возвращает пустую строку.
        """
        selected_elems = bs.select(selector)
        if selected_elems is not None and len(selected_elems) > 0:
            return '\n'.join(elem.get_text() for elem in selected_elems)
        return ''
    
    def get_content(self, website, path):
        """Извлекает содержимое страницы с заданным URL."""
        url = website.url + path
        bs = self.get_page(url)
        if bs is not None:
            title = self.safe_get(bs, website.title_tag)
            body = self.safe_get(bs, website.body_tag)
            return Content(url, title, body)
        return Content(url, '', '')


crawler = Crawler()

site_data = [
    ['O\'Reilly Media', 'https://www.oreilly.com', 'h1', 'div.title-description'],
    ['Reuters', 'https://www.reuters.com', 'h1', 'div.ArticleBodyWrapper'],
    ['Brookings', 'https://www.brookings.edu', 'h1', 'div.post-body'],
    ['CNN', 'https://www.cnn.com', 'h1', 'div.article__content']
]

websites = []

for name, url, title, body in site_data:
    websites.append(Website(name, url, title, body))
    
crawler.get_content(websites[0], '/library/view/web-scraping-with/9781491910283').print_info()
crawler.get_content(websites[1], '/article/us-usa-epa-pruitt-idUSKBN19W2D0').print_info()
crawler.get_content(websites[2], '/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/').print_info()
crawler.get_content(websites[3], '/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html').print_info()

URL: https://www.oreilly.com/library/view/web-scraping-with/9781491910283
TITLE: Web Scraping with Python
BODY:

URL: https://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0
TITLE: 
BODY:

URL: https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
TITLE: Idea to Retire: Old methods of policy education
BODY:

URL: https://www.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html
TITLE: 
      Dogecoin jumps after Elon Musk replaces Twitter bird with Shiba Inu
    
BODY:



New York
CNN
         — 
    


            Twitter’s traditional bird icon was booted and replaced with an image of a Shiba Inu, an apparent nod to dogecoin, the joke cryptocurrency that CEO Elon Musk is being sued over. 
    

            Musk addressed the change Monday afternoon, tweeting, “as promised” above an image of a year-old conversation in which another user suggested that Musk “just buy Twitter” and “change the bird logo to a doge.” 
    



In [116]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
image_location = bs.find_all('a')
for image in image_location:
    images = image.find('img')
    if images is None:
        continue
    print(images['src'])
    urlretrieve(images['src'], 'logo.jpg')

https://pythonscraping.com/wp-content/uploads/2021/08/logo01-e1681353135199.png


In [117]:
from urllib.request import urlretrieve, urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
image_location = bs.find('img', {'alt': 'python-logo'})['src']
urlretrieve (image_location, 'logo2.jpg')

('logo2.jpg', <http.client.HTTPMessage at 0x1ba273058d0>)

In [121]:
import os
from urllib.request import urlretrieve, urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup

download_dir = 'downloaded'
base_url = 'https://pythonscraping.com/'

def get_absolute_url(source):
    if urlparse(base_url).netloc == '':
        return base_url + source
    return source

def get_download_path(file_url):
    parsed = urlparse(file_url)
    netloc = parsed.netloc.strip('/')
    path = parsed.path.strip('/')
    localfile = f'{download_dir}/{netloc}/{path}'
    
    # Remove the filename from the path in order to 
    # make the directory structure leading up to it
    localpath = '/'.join(localfile.split('/')[:-1])
    if not os.path.exists(localpath):
        os.makedirs(localpath)
    return localfile

html = urlopen(base_url)
bs = BeautifulSoup(html, 'html.parser')
download_list = bs.find_all(src=True)

for download in download_list:
    file_url = get_absolute_url(download['src'])
    if file_url is not None:
        try:
            urlretrieve(file_url, get_download_path(file_url))
            print(file_url)
        except Exception as e:
            print(f'Could not retrieve {file_url} Error: {e}')

https://pythonscraping.com/wp-includes/js/jquery/jquery.min.js?ver=3.7.1
https://pythonscraping.com/wp-includes/js/jquery/jquery-migrate.min.js?ver=3.4.1
https://pythonscraping.com/wp-content/plugins/pagelayer/js/combined.js?ver=1.7.5
https://pythonscraping.com/wp-content/plugins/email-capture-lead-generation//js/eclg-public.js?ver=1.0.1
https://www.googletagmanager.com/gtag/js?id=GT-TNFZZK6
https://pythonscraping.com/wp-content/uploads/2023/04/python-logo-e1681354047443.png
https://pythonscraping.com/wp-content/uploads/2021/08/home1.jpg
https://pythonscraping.com/wp-content/uploads/2021/08/logo01-e1681353135199.png
https://pythonscraping.com/wp-content/plugins/email-capture-lead-generation//images/ajax_loader.gif
https://pythonscraping.com/wp-content/plugins/contact-form-7/includes/swv/js/index.js?ver=5.7.7
https://pythonscraping.com/wp-content/plugins/contact-form-7/includes/js/index.js?ver=5.7.7
https://pythonscraping.com/wp-content/themes/popularfx/js/navigation.js?ver=1.2.0


In [126]:
import csv

csv_file = open('test.csv', 'w+')
try:
    writer = csv.writer(csv_file, lineterminator="\n")
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow((i, i+2, i*2))
finally:
    csv_file.close()

In [137]:
import csv
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 Edg/142.0.0.0"}

html = requests.get('https://en.wikipedia.org/wiki/Comparison_of_text_editors', headers=headers)
bs = BeautifulSoup(html.text, 'html.parser')

# The main comparison table is currently the first table on the page
table = bs.find('table', {'class': 'wikitable'})
rows = table.find_all('tr')
csv_file = open('editors.csv', 'wt+', encoding='utf-8')
writer = csv.writer(csv_file)

try:
    for row in rows:
        csv_row = []
        for cell in row.find_all(['td', 'th']):
            csv_row.append(cell.get_text().strip())
        writer.writerow(csv_row)
finally:
    csv_file.close()

In [142]:
from urllib.request import urlopen

text_page = urlopen('http://www.pythonscraping.com/pages/warandpeace/chapter1-ru.txt')
print(str(text_page.read(), encoding='utf-8'))

ЧАСТЬ ПЕРВАЯ

I

— Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens que si vous ne me dites pas que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocités de cet Antichrist (ma parole, j'y crois) — je ne vous connais plus, vous n'êtes plus mon ami, vous n'êtes plus мой верный раб, comme vous dites. Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, садитесь и рассказывайте.
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех:
«Si vous n'avez rien de mieux à faire, Monsieur le comte (или mon 

In [3]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 Edg/142.0.0.0"}

html = requests.get("http://en.wikipedia.org/wiki/Python_(programming_language)", headers=headers)
bs = BeautifulSoup(html.text, "html.parser")

content = bs.find("div", {"id": "mw-content-text"}).get_text()
content = bytes(content, "UTF-8")
content = content.decode("UTF-8")
print(content)

This article has multiple issues. Please help improve it or discuss these issues on the talk page. (Learn how and when to remove these messages)

This article needs to be updated. The reason given is: Information related to stable Python version 3.14.0 is not present. Please help update this article to reflect recent events or newly available information. (October 2025)This article may contain unverified or indiscriminate information in embedded lists. Please help clean up the lists by removing items or incorporating them into the text of the article. (October 2025)

 (Learn how and when to remove this message)General-purpose programming language


PythonParadigmMulti-paradigm: object-oriented,[1] procedural (imperative), functional, structured, reflectiveDesigned byGuido van RossumDeveloperPython Software FoundationFirst appeared20 February 1991; 34 years ago (1991-02-20)[2]Stable release3.14.0[3] 
   / 7 October 2025; 38 days ago (7 October 2025)
Typing disciplineDuck, dynamic, stron

In [6]:
from urllib.request import urlopen
from io import StringIO
import csv

data = urlopen('http://pythonscraping.com/files/MontyPythonAlbums.csv').read().decode('ascii', 'ignore')
data_file = StringIO(data)
csv_reader = csv.reader(data_file)
for row in csv_reader:
    print(f'The album "{row[0]}" was released in {row[1]}')

The album "Name" was released in Year
The album "Monty Python's Flying Circus" was released in 1970
The album "Another Monty Python Record" was released in 1971
The album "Monty Python's Previous Record" was released in 1972
The album "The Monty Python Matching Tie and Handkerchief" was released in 1973
The album "Monty Python Live at Drury Lane" was released in 1974
The album "An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail" was released in 1975
The album "Monty Python Live at City Center" was released in 1977
The album "The Monty Python Instant Record Collection" was released in 1977
The album "Monty Python's Life of Brian" was released in 1979
The album "Monty Python's Cotractual Obligation Album" was released in 1980
The album "Monty Python's The Meaning of Life" was released in 1983
The album "The Final Rip Off" was released in 1987
The album "Monty Python Sings" was released in 1989
The album "The Ultimate Monty Python Rip Off" was release

In [7]:
from urllib.request import urlopen
from io import StringIO
import csv

data = urlopen('http://pythonscraping.com/files/MontyPythonAlbums.csv').read().decode('ascii', 'ignore')
data_file = StringIO(data)
dict_reader = csv.DictReader(data_file)
print(dict_reader.fieldnames)

for row in dict_reader:
    print(row)

['Name', 'Year']
{'Name': "Monty Python's Flying Circus", 'Year': '1970'}
{'Name': 'Another Monty Python Record', 'Year': '1971'}
{'Name': "Monty Python's Previous Record", 'Year': '1972'}
{'Name': 'The Monty Python Matching Tie and Handkerchief', 'Year': '1973'}
{'Name': 'Monty Python Live at Drury Lane', 'Year': '1974'}
{'Name': 'An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail', 'Year': '1975'}
{'Name': 'Monty Python Live at City Center', 'Year': '1977'}
{'Name': 'The Monty Python Instant Record Collection', 'Year': '1977'}
{'Name': "Monty Python's Life of Brian", 'Year': '1979'}
{'Name': "Monty Python's Cotractual Obligation Album", 'Year': '1980'}
{'Name': "Monty Python's The Meaning of Life", 'Year': '1983'}
{'Name': 'The Final Rip Off', 'Year': '1987'}
{'Name': 'Monty Python Sings', 'Year': '1989'}
{'Name': 'The Ultimate Monty Python Rip Off', 'Year': '1994'}
{'Name': 'Monty Python Sings Again', 'Year': '2014'}


In [8]:
from urllib.request import urlopen
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO, open

def read_pdf(pdf_file):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    process_pdf(rsrcmgr, device, pdf_file)
    device.close()
    content = retstr.getvalue()
    retstr.close()
    return content

pdf_file = urlopen('http://pythonscraping.com/pages/warandpeace/chapter1.pdf')
output_string = read_pdf(pdf_file)
print(output_string)
pdf_file.close()


CHAPTER I

"Well, Prince, so Genoa and Lucca are now just family estates of 
theBuonapartes. But I warn you, if you don't tell me that this 
means war,if you still try to defend the infamies and horrors 
perpetrated bythat Antichrist- I really believe he is Antichrist- I will 
havenothing more to do with you and you are no longer my friend, 
no longermy 'faithful slave,' as you call yourself! But how do you 
do? I seeI have frightened you- sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known 
AnnaPavlovna Scherer, maid of honor and favorite of the 
Empress MaryaFedorovna. With these words she greeted Prince 
Vasili Kuragin, a manof high rank and importance, who was the 
first to arrive at herreception. Anna Pavlovna had had a cough for 
some days. She was, asshe said, suffering from la grippe; grippe 
being then a new word inSt. Petersburg, used only by the elite.

All her invitations without exception, written in French, 
anddelivered by a scar

In [11]:
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
from bs4 import BeautifulSoup

word_file = urlopen('http://pythonscraping.com/pages/AWordDocument.docx').read()
word_file = BytesIO(word_file)
document = ZipFile(word_file)
xml_content = document.read('word/document.xml')
word_obj = BeautifulSoup(xml_content.decode('utf-8'), 'xml')
text_strings = word_obj.find_all('w:t')

for text_elem in text_strings:
    style = text_elem.parent.parent.find('w:pStyle')
    if style is not None and style['w:val'] == 'Title':
        print(f'Title is: {text_elem.text}')
    else:
        print(text_elem.text)

A Word Document on a Website
This is a Word document, full of content that you want very much. 
Unfortuna
t
ly
, it’s difficult to access because I’m putting it on my website as a .docx file, rather than just publishing it as HTML. 


In [15]:
import re
import string 
import unicodedata
import requests
from bs4 import BeautifulSoup

# Must be called before split_sentences
def replace_newlines(text):
    return text.replace('\n', ' ')

def make_lowercase(text):
    return text.lower()

def strip_citations(text):
    CITATION_REGEX = re.compile('\[[0-9]*\]')
    return re.sub(CITATION_REGEX, '', text)


def split_sentences(text):
    return [s.strip() for s in text.split('. ')]

def remove_parentheses(text):
    PARENS_REGEX = re.compile('\([a-z A-Z \+\.,\-]{0,100}\)')
    return re.sub(PARENS_REGEX, '', text)


def remove_descriptions(text):
    DESCRIPTION_REGEX = re.compile('\n[a-z A-Z]*:')
    return re.sub(DESCRIPTION_REGEX, '', text)


def remove_punctuation(text):
    puncts = [re.escape(c) for c in string.punctuation]
    PUNCTUATION_REGEX = re.compile('|'.join(puncts))
    return re.sub(PUNCTUATION_REGEX, '', text)

def normalize(text):
    return unicodedata.normalize('NFKD', text)

text_operations = [strip_citations, remove_parentheses, remove_descriptions,
                   replace_newlines, split_sentences, make_lowercase,
                   remove_punctuation, normalize]

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36 Edg/142.0.0.0"}

html = requests.get('http://en.wikipedia.org/wiki/Python_(programming_language)', headers=headers)
bs = BeautifulSoup(html.text, 'html.parser')
content = bs.find('div', {'id': 'mw-content-text'}).get_text()

cleaned = content
for op in text_operations:
    if type(cleaned) == list:
        cleaned = [op(c) for c in cleaned]
    else:
        cleaned = op(cleaned)

print(cleaned)
print(f'2-grams count is: {len(cleaned)}')
    

['this article has multiple issues', 'please help improve it or discuss these issues on the talk page', 'this article needs to be updated', 'the reason given is information related to stable python version 3140 is not present', 'please help update this article to reflect recent events or newly available information', 'october 2025this article may contain unverified or indiscriminate information in embedded lists', 'please help clean up the lists by removing items or incorporating them into the text of the article', 'october 2025   generalpurpose programming language   pythonparadigmmultiparadigm objectoriented procedural  functional structured reflectivedesigned byguido van rossumdeveloperpython software foundationfirst appeared20 february 1991 34 years ago 19910220stable release3140      7 october 2025 38 days ago 7 october 2025 typing disciplineduck dynamic strong optional type annotationsaoscrossplatform including 32bit windows 10 and eg', 'for mobile android and ios both 64bitblice